In [1]:
import pennylane as qml
from pennylane import numpy as np
import qemzmsepcV2 as qem

In [2]:
n_qubits = 4
dev = qml.device('default.mixed', wires=n_qubits)
rotation_angle_of_rx = 0
rotation_angle_of_rz = -0.2

def trotterstep1():
    qml.RX(rotation_angle_of_rx, wires=0)
    qml.RX(rotation_angle_of_rx, wires=1)
    qml.RX(rotation_angle_of_rx, wires=2)
    qml.RX(rotation_angle_of_rx, wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])
    qml.RZ(rotation_angle_of_rz, wires=1)
    qml.RZ(rotation_angle_of_rz, wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[1, 2])
    qml.RZ(rotation_angle_of_rz, wires=2)
    qml.CNOT(wires=[1, 2])

In [3]:
nqubitschannel = qem.NqubitsChannel(n_qubits)
pauli_channel = nqubitschannel.nqubitsrandompaulichannel(p_identity=0.5)
qemzmsepc = qem.QEMZMSEPC(n_qubits)

rotation_angle = [0, 0.5, 0.9, 1.3, 1.7, 2.1, 2.5]
p = 0.8
for rotation_angle_of_rx in rotation_angle:
    z_ideal = qemzmsepc.noise_circuit_output(dev, trotterstep1)
    print(f"z_ideal is {z_ideal}")
    z_unmitigated = qemzmsepc.noise_circuit_output(dev, trotterstep1, p=p,
                       kraus_matrices_of_a_pauli_channel=pauli_channel,
                       need_gate_noise=True, need_measurement_noise=True)
    print(f"z_unmitigated is {z_unmitigated}")
    z_mitigated, _ = qemzmsepc.qemzmsepc(trotterstep1, p, dev, pauli_channel)
    print(f"z_mitigated is {z_mitigated}")
    print("------------------")


z_ideal is 1.0
z_unmitigated is 0.44885978569304075
z_mitigated is 1.0000000000000058
------------------
z_ideal is 0.5931327983656769
z_unmitigated is 0.26623346076193155
z_mitigated is 0.5931327983656792
------------------
z_ideal is 0.1493041506116879
z_unmitigated is 0.06701662904664486
z_mitigated is 0.1493041506116927
------------------
z_ideal is 0.0051202072280735166
z_unmitigated is 0.0022982551190989277
z_mitigated is 0.005120207228077064
------------------
z_ideal is 0.0002755900039979539
z_unmitigated is 0.00012370127013251048
z_mitigated is 0.00027559000399602496
------------------
z_ideal is 0.06495850756506487
z_unmitigated is 0.02915726178459092
z_mitigated is 0.0649585075650566
------------------
z_ideal is 0.4119471515970569
z_unmitigated is 0.1849065101827162
z_mitigated is 0.4119471515970647
------------------
